In [1]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
from scipy.stats import shapiro, normaltest
import os
from pathlib import Path

import pingouin as pg

warnings.filterwarnings('ignore')
probe_ord = ['NNXr', 'NNXo']
#hue_ord = [NNXR_GRAY, MAIN_COLOR]

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/Users/driessen2@ad.wisc.edu/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *
import pubplots as pp
plt.rcdefaults()
plt.style.use('/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr/src/acr/plot_styles/acrvec_labels.mplstyle')

/Users/driessen2@ad.wisc.edu/kdriessen/acr_dev/acr_revs/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nbroot = os.path.join(PAPER_FIGURE_ROOT, 'response_to_review', 'haloN')
Path(nbroot).mkdir(parents=True, exist_ok=True)

In [3]:
from acr.stats import replication_bayes_factor

# SWA in Rebound

In [4]:
COMP_ANALYSIS = 'swa_reb'
results = {}

In [5]:
SUBJECT_TYPE = 'som'
MAIN_EXP = 'swi'
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
scon = src['contra_control'].values
sop = src['off_induction'].values
results[SUBJECT_TYPE] = sop - scon

In [6]:
SUBJECT_TYPE = 'acr'
MAIN_EXP = 'swi'
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
scon = src['contra_control'].values
sop = src['off_induction'].values
results[SUBJECT_TYPE] = sop - scon

In [7]:
SUBJECT_TYPE = 'halo'
MAIN_EXP = 'swisin'
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
scon = src['contra_control'].values
sop = src['off_induction'].values
results[SUBJECT_TYPE] = sop - scon

In [8]:
deltas_off = np.concatenate([results['som'], results['acr']])
deltas_halo = results['halo']
# Compute replication Bayes factor
result = replication_bayes_factor(deltas_off, deltas_halo)

print("OFF group (ACR+SOM) mean ΔSWA:", result.mu0)
print("HALO group mean ΔSWA:", result.mu_hat_halo)
print("BF_0r (H0: no effect vs Hr: OFF-like):", result.bf_0r)
print("BF_r0 (Hr vs H0):", result.bf_r0)
print("Posterior under Hr given HALO data: μ = {:.3f} ± {:.3f}".format(
    result.mu_post, result.sd_post
))

OFF group (ACR+SOM) mean ΔSWA: -0.19070914736842107
HALO group mean ΔSWA: 0.024989983333333354
BF_0r (H0: no effect vs Hr: OFF-like): 1202.586443967751
BF_r0 (Hr vs H0): 0.0008315410547125845
Posterior under Hr given HALO data: μ = -0.120 ± 0.027


In [9]:
from scipy.stats import norm

mu_off = result.mu0
mu_post = result.mu_post
sd_post = result.sd_post

# P(|mu_halo| >= |mu_off| | HALO data, Hr)
# For symmetric Normal we can do it two-sided:
threshold = abs(mu_off)

p_geq = 1 - (norm.cdf(threshold, loc=mu_post, scale=sd_post)
             - norm.cdf(-threshold, loc=mu_post, scale=sd_post))

print("P(|mu_halo| >= |mu_off| | HALO data, Hr) =", p_geq)

P(|mu_halo| >= |mu_off| | HALO data, Hr) = 0.004552059005464626


# STTC

In [10]:
COMP_ANALYSIS = 'sttc'
results = {}

In [11]:
src_dats = []

In [12]:
SUBJECT_TYPE = 'som'
MAIN_EXP = 'swi'
cons = []
ops = []
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [13]:
SUBJECT_TYPE = 'acr'
MAIN_EXP = 'swi'
cons = []
ops = []
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
subjects, exps = remove_subject('ACR_16', subjects, exps)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [14]:
SUBJECT_TYPE = 'halo'
MAIN_EXP = 'swisin'
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [15]:
deltas_off = np.concatenate([results['som'], results['acr']])
deltas_halo = results['halo']
# Compute replication Bayes factor
result = replication_bayes_factor(deltas_off, deltas_halo)

print("OFF group (ACR+SOM) mean ΔSWA:", result.mu0)
print("HALO group mean ΔSWA:", result.mu_hat_halo)
print("BF_0r (H0: no effect vs Hr: OFF-like):", result.bf_0r)
print("BF_r0 (Hr vs H0):", result.bf_r0)
print("Posterior under Hr given HALO data: μ = {:.3f} ± {:.3f}".format(
    result.mu_post, result.sd_post
))

mu_off = result.mu0
mu_post = result.mu_post
sd_post = result.sd_post

# P(|mu_halo| >= |mu_off| | HALO data, Hr)
# For symmetric Normal we can do it two-sided:
threshold = abs(mu_off)

p_geq = 1 - (norm.cdf(threshold, loc=mu_post, scale=sd_post)
             - norm.cdf(-threshold, loc=mu_post, scale=sd_post))

print("P(|mu_halo| >= |mu_off| | HALO data, Hr) =", p_geq)

OFF group (ACR+SOM) mean ΔSWA: -0.01911404128944929
HALO group mean ΔSWA: -0.0015134548251060996
BF_0r (H0: no effect vs Hr: OFF-like): 3.995287052619746
BF_r0 (Hr vs H0): 0.2502949066811835
Posterior under Hr given HALO data: μ = -0.016 ± 0.004
P(|mu_halo| >= |mu_off| | HALO data, Hr) = 0.24068014219101053


In [16]:
deltas_off = results['acr']
deltas_halo = results['halo']
# Compute replication Bayes factor
result = replication_bayes_factor(deltas_off, deltas_halo)

print("OFF group (ACR+SOM) mean ΔSWA:", result.mu0)
print("HALO group mean ΔSWA:", result.mu_hat_halo)
print("BF_0r (H0: no effect vs Hr: OFF-like):", result.bf_0r)
print("BF_r0 (Hr vs H0):", result.bf_r0)
print("Posterior under Hr given HALO data: μ = {:.3f} ± {:.3f}".format(
    result.mu_post, result.sd_post
))

mu_off = result.mu0
mu_post = result.mu_post
sd_post = result.sd_post

# P(|mu_halo| >= |mu_off| | HALO data, Hr)
# For symmetric Normal we can do it two-sided:
threshold = abs(mu_off)

p_geq = 1 - (norm.cdf(threshold, loc=mu_post, scale=sd_post)
             - norm.cdf(-threshold, loc=mu_post, scale=sd_post))

print("P(|mu_halo| >= |mu_off| | HALO data, Hr) =", p_geq)

OFF group (ACR+SOM) mean ΔSWA: -0.01198893888337985
HALO group mean ΔSWA: -0.0015134548251060996
BF_0r (H0: no effect vs Hr: OFF-like): 1.7172919111053162
BF_r0 (Hr vs H0): 0.5823121820659837
Posterior under Hr given HALO data: μ = -0.010 ± 0.004
P(|mu_halo| >= |mu_off| | HALO data, Hr) = 0.32414038697370273


# Init_slope

In [17]:
COMP_ANALYSIS = 'rebound_term'
results = {}

In [18]:
src_dats = []

In [35]:
SUBJECT_TYPE = 'som'
MAIN_EXP = 'swi'
cons = []
ops = []
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [36]:
SUBJECT_TYPE = 'acr'
MAIN_EXP = 'swi'
cons = []
ops = []
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
subjects, exps = remove_subject('ACR_16', subjects, exps)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [37]:
SUBJECT_TYPE = 'halo'
MAIN_EXP = 'swisin'
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
src = pd.read_csv(f'./{COMP_ANALYSIS}/{SUBJECT_TYPE}.csv')
src['subject'] = subjects
locations = [acr.utils.sub_probe_locations[sub] for sub in subjects]
src['location'] = locations
src['sub_type'] = SUBJECT_TYPE
scon = src['contra_control'].values
cons.append(scon)
sop = src['off_induction'].values
ops.append(sop)
results[SUBJECT_TYPE] = sop - scon
src_dats.append(src)

In [38]:
deltas_off = np.concatenate([results['som'], results['acr']])
deltas_halo = results['halo']
# Compute replication Bayes factor
result = replication_bayes_factor(deltas_off, deltas_halo)

print("OFF group (ACR+SOM) mean ΔSWA:", result.mu0)
print("HALO group mean ΔSWA:", result.mu_hat_halo)
print("BF_0r (H0: no effect vs Hr: OFF-like):", result.bf_0r)
print("BF_r0 (Hr vs H0):", result.bf_r0)
print("Posterior under Hr given HALO data: μ = {:.3f} ± {:.3f}".format(
    result.mu_post, result.sd_post
))

mu_off = result.mu0
mu_post = result.mu_post
sd_post = result.sd_post

# P(|mu_halo| >= |mu_off| | HALO data, Hr)
# For symmetric Normal we can do it two-sided:
threshold = abs(mu_off)

p_geq = 1 - (norm.cdf(threshold, loc=mu_post, scale=sd_post)
             - norm.cdf(-threshold, loc=mu_post, scale=sd_post))

print("P(|mu_halo| >= |mu_off| | HALO data, Hr) =", p_geq)

OFF group (ACR+SOM) mean ΔSWA: -0.13896590151267443
HALO group mean ΔSWA: -0.06166466000000006
BF_0r (H0: no effect vs Hr: OFF-like): 1.3044066967036188
BF_r0 (Hr vs H0): 0.7666320653881274
Posterior under Hr given HALO data: μ = -0.125 ± 0.025
P(|mu_halo| >= |mu_off| | HALO data, Hr) = 0.27980865050867876


In [39]:
deltas_off = results['acr']
deltas_halo = results['halo']
# Compute replication Bayes factor
result = replication_bayes_factor(deltas_off, deltas_halo)

print("OFF group (ACR+SOM) mean ΔSWA:", result.mu0)
print("HALO group mean ΔSWA:", result.mu_hat_halo)
print("BF_0r (H0: no effect vs Hr: OFF-like):", result.bf_0r)
print("BF_r0 (Hr vs H0):", result.bf_r0)
print("Posterior under Hr given HALO data: μ = {:.3f} ± {:.3f}".format(
    result.mu_post, result.sd_post
))

mu_off = result.mu0
mu_post = result.mu_post
sd_post = result.sd_post

# P(|mu_halo| >= |mu_off| | HALO data, Hr)
# For symmetric Normal we can do it two-sided:
threshold = abs(mu_off)

p_geq = 1 - (norm.cdf(threshold, loc=mu_post, scale=sd_post)
             - norm.cdf(-threshold, loc=mu_post, scale=sd_post))

print("P(|mu_halo| >= |mu_off| | HALO data, Hr) =", p_geq)

OFF group (ACR+SOM) mean ΔSWA: -0.14861092396231174
HALO group mean ΔSWA: -0.06166466000000006
BF_0r (H0: no effect vs Hr: OFF-like): 1.426554037850832
BF_r0 (Hr vs H0): 0.7009899193910278
Posterior under Hr given HALO data: μ = -0.110 ± 0.038
P(|mu_halo| >= |mu_off| | HALO data, Hr) = 0.155505304021473
